In [1]:
import tqdm
import numpy as np
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTMCell, RNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

2023-12-01 01:41:18.596903: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-01 01:41:18.628218: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 01:41:18.778028: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 01:41:18.781118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 01:41:19.826037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd "/content/drive/MyDrive/College/Sem 5/SMAI/Project"

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU is detected")

No GPU is detected


2023-12-01 01:41:22.042954: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
import random

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)


In [6]:
# !pip install datasets

In [7]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")


Found cached dataset yelp_review_full (/home/sriteja/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [9]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [10]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [12]:
with open('yelp_train_tokens_no_stop.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

with open('yelp_test_tokens_no_stop.pkl', 'rb') as file:
    test_tokens = pickle.load(file)

In [13]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'


In [14]:
tokens = train_tokens
tokens_test = test_tokens
print(len(tokens))
print(len(tokens_test))

650000
50000


In [15]:
# find length of longest sentence
max_len = 0
for i in range(len(tokens)):
    if len(tokens[i]) > max_len:
        max_len = len(tokens[i])

# find length of shortest sentence
min_len = 1000000
for i in range(len(tokens)):
    if len(tokens[i]) < min_len:
        min_len = len(tokens[i])

print("Max length: ", max_len)
print("Min length: ", min_len)

# number of sentences with length > 100
count = 0
for i in range(len(tokens)):
    if len(tokens[i]) > 250:
        count += 1

print("Number of sentences with length > 250: ", count)

Max length:  676
Min length:  0
Number of sentences with length > 250:  11454


In [16]:
# remove sentences with length > 100 along with their labels
new_tokens = []
new_labels = []
for i in range(len(tokens)):
    if len(tokens[i]) <= 250:
        new_tokens.append(tokens[i])
        new_labels.append(train_labels[i])

# remove so in test also
new_tokens_test = []
new_labels_test = []
for i in range(len(tokens_test)):
    if len(tokens_test[i]) <= 250:
        new_tokens_test.append(tokens_test[i])
        new_labels_test.append(test_labels[i])

print(len(new_tokens))
print(len(new_labels))
print(len(new_tokens_test))
print(len(new_labels_test))

638546
638546
49123
49123


In [17]:
# for tetst data
# find length of longest sentence
max_len_test = 0
for i in range(len(tokens_test)):
    if len(tokens_test[i]) > max_len_test:
        max_len_test = len(tokens_test[i])

# find length of shortest sentence
min_len_test = 1000000
for i in range(len(tokens_test)):
    if len(tokens_test[i]) < min_len_test:
        min_len_test = len(tokens_test[i])

print("Max length: ", max_len_test)
print("Min length: ", min_len_test)

Max length:  497
Min length:  0


In [18]:
# add padding to all sentences and S and EOS tokens
def padding(tokens_list, max_len):
    for i in range(len(tokens_list)):
        tokens_list[i] = ['S'] + tokens_list[i] + ['EOS']
        while len(tokens_list[i]) < max_len:
            tokens_list[i].append('PAD')
    return tokens_list

In [19]:
max_len = 252
tokens = padding(new_tokens, max_len)
tokens_test = padding(new_tokens_test, max_len)

In [20]:
print(len(tokens))
print(len(tokens_test))

638546
49123


In [21]:
# split into train and validation using sklearn
from sklearn.model_selection import train_test_split
tokens, val_tokens, new_labels, val_labels = train_test_split(tokens, new_labels, test_size=0.1, random_state=42)


In [22]:
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for token in tokens:
    vocab.update(token)

# Create a dictionary to map words to indices in the vocabulary
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for idx, word in enumerate(vocab)}

print(idx_to_word[0])
print(word_to_idx['UNK'])
print(word_to_idx['PAD'])
print(word_to_idx['S'])
print(word_to_idx['EOS'])


tantlizing
126930
13646
110297
82486


In [23]:


# Convert tokens and labels to Keras format
# Note: Ensure tokens and new_labels are numpy arrays
X_train = [[word_to_idx.get(token, word_to_idx['UNK']) for token in sentence] for sentence in tokens]
X_val = [[word_to_idx.get(token, word_to_idx['UNK']) for token in sentence] for sentence in val_tokens]
X_test = [[word_to_idx.get(token, word_to_idx['UNK']) for token in sentence] for sentence in tokens_test]
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

num_classes = 5

# Convert labels to one-hot format
y_train = to_categorical(new_labels, num_classes=num_classes)
y_val = to_categorical(val_labels, num_classes=num_classes)
y_test = to_categorical(new_labels_test, num_classes=num_classes)

In [ ]:
# # Model
# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True))
# model.add(RNN(LSTMCell(hidden_size)))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(optimizer=Adam(learning_rate=learning_rate),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [24]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTMCell, RNN, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: silvertongue1729. Use `wandb login --relogin` to force relogin


True

In [25]:
batch_size = 256
vocab_size = len(vocab)
num_epochs = 1
learning_rate = 0.001
embedding_size = 300
hidden_size = 128

In [26]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'   
    },
    'parameters': {
        'embedding_size': {'values': [100, 300]},
        'hidden_size': {'values': [64, 128, 256]},
        'learning_rate': {'values': [0.001, 0.0001]},
        'batch_size': {'values': [256]},
        'num_epochs': {'values': [1, 2]}
    },
    "name": "lstm_yelp_sweep"
}


In [27]:
def train():
    # Initialize a wandb run for each sweep iteration
    with wandb.init() as run:
        config = wandb.config

        print("Config: ", config)

        # Model architecture
        model = Sequential()
        model.add(Embedding(input_dim=vocab_size, output_dim=config.embedding_size, mask_zero=True))
        model.add(RNN(LSTMCell(hidden_size)))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=Adam(learning_rate=config.learning_rate),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

        # Training the model
        history = model.fit(X_train, y_train, epochs=config.num_epochs, batch_size=config.batch_size, shuffle=True, 
                            validation_data=(X_val, y_val), callbacks=[WandbCallback()])



In [ ]:
sweep_id = wandb.sweep(sweep_config, project="SMAI_project_lstm_yelp")
wandb.agent(sweep_id, train, count=10)

In [ ]:
import sys
sys.exit()

In [ ]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTMCell, RNN, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Initialize wandb run
wandb.login()
wandb.init(project="SMAI_project_lstm_yelp", name="lstm_yelp_model_run_1")

# Configurations
wandb.config.embedding_size = embedding_size
wandb.config.hidden_size = hidden_size
wandb.config.learning_rate = learning_rate
wandb.config.batch_size = batch_size
wandb.config.num_epochs = num_epochs

# Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True))
model.add(RNN(LSTMCell(hidden_size)))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, shuffle=True,
                    validation_data=(X_val, y_val), callbacks=[WandbCallback()])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
test_accuracy = np.mean(y_pred_classes == y_test_labels) * 100

# Log additional metrics
wandb.log({'Test Accuracy': test_accuracy})
print('Default Test Accuracy:', test_accuracy)

# Classification report and confusion matrix
report = classification_report(y_test_labels, y_pred_classes)
matrix = confusion_matrix(y_test_labels, y_pred_classes)
print(report)
print(matrix)

# Optionally log classification report and confusion matrix
wandb.log({"Classification Report": report, "Confusion Matrix": matrix})

# Finish the wandb run
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: silvertongue1729. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


1473/2245 [==================>...........] - ETA: 12:24 - loss: 1.6097 - accuracy: 0.2014